# Workshop Deep Learning

Dieses Tutorial zeigt, wie man mittels eines Neuronalen Netzwerks / Deep Learning einen Bild-Klassifizierer baut, der Katzenbilder von Hundebildern unterscheidet.

Diese Aufgabenstellung kommt aus dem ["Cats vs. Dogs"](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition) Wettbewerb der Website Kaggle.

Basis bildet ein Convolutional Neuronal Network (CNN) namens "VGG16", welches auf Basis der Daten des [Imagenet Datasets](http://image-net.org/synset?wnid=n02084071) vortrainiert wurde. Das Modell wird durch Umkonfiguration und Re-Training so angepasst, dass es die gestellte Aufgabe lösen kann.

Die Grundlagen zu diesem Workshop kommen aus dem Deep Learning MOOC [fast.ai](http://fast.ai).

## Inhalt

1. [Data preparation](#Data-preparation) - Welche Daten verarbeiten wir?
1. [Setup](#Setup) - Initialisierung der notwendigen Frameworks und des Modells
1. [Training](#Training) - Training des Modells mit unseren Daten
1. [Vorhersage](#Vorhersage) - Das trainierte Modell anwenden, um eine Vorhersage zu treffen
1. [Visualisieren der Ergebnisse](#Visualisieren-der-Ergebnisse) - Wir schauen uns an, welche Ergebnisse das Modell liefert
1. [Mehr Layer trainieren](#Mehr-Layer-trainieren) - Das Training intensivieren
1. [Exkurs: Wenige Bilder Trainieren, viele Bilder klassifizieren](#Exkurs:-Wenige-Bilder-Trainieren,-viele-Bilder-klassifizieren)


## Data preparation

Die Daten des Kaggle Wettbewerbs wurden schon vorbereitet und in der "richtigen" Struktur abgelegt.
Das Verzeichnis `data` enthält die Trainings- und Validierungsdaten aus dem Dataset. Dabei sind die Bilder zu jeder zu erkennenden "Klasse" (Cats & Dogs in unserem Fall) in einem eigenen Unterverzeichnis abgelegt:

In [ ]:
# ! führt einen Shell-Befehl aus...
!tree -d data

Mal schauen, wieviele Dateien in den Trainings- und Validerungsdaten drin sind:

In [ ]:
!echo -n "Training cats: " && ls data/train/cats | wc -w
!echo -n "Training dogs: " && ls data/train/dogs | wc -w
!echo -n "Validation cats: " && ls data/valid/cats | wc -w
!echo -n "Validation dogs: " && ls data/valid/dogs | wc -w

Das Verzeichnis `test` enthält die Bilder, die nicht klassifizert sind (deshalb das Unterverzeichnis `unknown`). Diese wollen wir nach dem Training bestimmen. Mal sehen, wieviele das sind:

In [ ]:
!echo -n "Test: " && ls data/test/unknown | wc -w

Damit wir am Code herumprobieren können, ohne gleich lange Laufzeiten aufgrund der vielen Dateien zu erhalten, gibt es noch ein `sample` Dataset, welches gleich aufgebaut ist, aber nur nur einen kleinen Teil der Daten enthält:

In [ ]:
!tree -d sample

In [ ]:
!echo -n "Training cats: " && ls sample/train/cats | wc -w
!echo -n "Training dogs: " && ls sample/train/dogs | wc -w
!echo -n "Validation cats: " && ls sample/valid/cats | wc -w
!echo -n "Validation dogs: " && ls sample/valid/dogs | wc -w
!echo -n "Test: " && ls sample/test/unknown | wc -w

## Setup

Diesen Teil immer ausführen. Hier werden notwendige Packete geladen und globale Variablen initialisiert.

In [ ]:
%matplotlib inline

from glob import glob
import shutil
import os.path
import time

import numpy as np
np.set_printoptions(precision=4, linewidth=100)
import utils
import keras
import sklearn

Hier setzen wir den Pfad für die Daten, mit denen wir arbeiten wollen (also `data` oder `sample`):

In [ ]:
# path = "data/"
path = "sample/"
path = os.path.join(os.path.curdir,path)
print path

train_path = os.path.join(path,"train")
valid_path = os.path.join(path,"valid")
test_path = os.path.join(path,"test")
result_path = os.path.join(path,"results")

# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

print train_path
print valid_path
print test_path
print result_path

## Training

Wir laden die Python Klasse, welche das hier verwendete Modell in ein nettes, mehr oder weniger objektorientiertes API verpackt. Der Sourcecode dazu steht in der Datei `vgg16.py`.

In [ ]:
# Import VGG16 class, and instantiate
import vgg16
vgg = vgg16.Vgg16()

Das VGG16 Modell ist ein Convolutional Neuronal Network (CNN) und wurde an der University of Oxford von der [Visual Geometry Group](http://www.robots.ox.ac.uk/~vgg/) entwickelt und veröffentlicht. Es wurde verwendet um den [Imagenet Contest](http://image-net.org/synset?wnid=n02084071) zu gewinnen. Es erkennt 1000 verschiedene Ojekte (= es liefert zu einem Bild 1000 Wahrscheinlichkeitswerte, ob das Bild ein Ding dieser Klasse enthält).

Wir können einen Blick in den Code der Klasse werfen (?? zeigt die Implementierung eines Code-Elements):

In [ ]:
??vgg

Wir können auch einen Blick auf die Struktur des Modells werfen. Der folgende Befehl zeigt die Schichten ("Layer"):

In [ ]:
vgg.model.summary()

Wir laden die Trainings- und Validierungsdaten als "Batches". Ein Batch liefert immer die nächsten _n_ Datensätze inklusive der Kategorie. So wird beim Training über die gesamte Menge der Trainings- und Validierungsdaten iteriert.

`vgg.get_batches()` erwartet, dass die Daten in Unterverzeichnissen je Kategorie abgelegt sind. Genau das ist bei uns der Fall, wie wir oben gesehen haben (Verzeichnisse "cats" und "dogs").

In [ ]:
train_batches = vgg.get_batches(train_path, batch_size=batch_size)
validation_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)


Wie wir auch sehen, hat der Befehl automatisch erkannt, wie viele Klassen in unseren Trainings-/Validierungsdaten enthalten sind: `2 classes` (cats & cogs).

Dann wird das VGG16 Modell an unsere Aufgabe ("cat or dog" Klassifizierung) angepasst mit `vgg.finetune()`. Dies ändert die Architektur des Netzes: Der letzte Layer wird verworfen und durch einen neuen Layer erstetzt, welcher nur noch 2 Outputs hat (statt wie bisher 1000): Cats & Dogs!

In [ ]:
vgg.finetune(train_batches)

Wir können uns anschauen, was die Methode `finetune` macht:

In [ ]:
??vgg.finetune

Der eigentliche Austausch des letzten Layers erfolgt in der Methode `vgg.ft()`. Die Gewichte des neuen Layers sind zunächst mit Zufallswerten initialisiert worden, d.h. sie müssen noch trainiert werden. Die anderen Layer lassen wir, wie sie sind (`layer.trainable = False`).

In [ ]:
??vgg.ft

Jetzt trainieren wir das Modell mit den Daten über `vgg.fit()`. Dabei wird in Wahrheit nur noch der letze (modifizierte) Layer des angepassten VGG16 Modells trainiert. Die angepassten Gewichte des Modells schreiben wir in eine Datei, so dass wir sie später wieder laden können und so nicht jedesmal das Training wiederholen müssen.

In [ ]:
# Learning rate:
vgg.model.optimizer.lr = 0.01

print 'start fitting at ' + time.asctime()

vgg.fit(train_batches, validation_batches, nb_epoch=1)

print 'stop fitting at ' + time.asctime()

weights_filename = os.path.join(result_path,'finetune1.h5')
print 'saving weights to ' + weights_filename
vgg.model.save_weights(weights_filename)

## Vorhersage

Jetzt klassifizieren wir mit dem re-trainierten Modell die Bilder, die im Unterverzeichnis 'test' abgelegt sind. Anders ausgedrückt: Wir sagen für eine Menge Daten (= ein Bild) vorher, mit jeweils welcher Wahrscheinlichkeit diese Daten eine Katze bzw. ein Hund ist. Die Ergebnisse speichern wir wieder in Dateien, damit wir später bei Bedarf darauf zugreifen können, ohne das ganze Training wiederholen zu müssen.

In [ ]:
# Dateien zum Speichern der Ergebnisse:
predictions_file = os.path.join(result_path,'predictions.dat')
filenames_file = os.path.join(result_path,'filenames.dat')

Prediction durchführen und Ergebnisse speichern:

In [ ]:
print 'start predicting at ' + time.asctime()
test_batches, predictions = vgg.test(test_path,batch_size=batch_size*2)
print 'stop predicting at '+ time.asctime()

filenames = test_batches.filenames
utils.save_array(predictions_file, predictions)
utils.save_array(filenames_file, filenames)

### Mal ein paar Ergebnisse anschauen

Zunächst eine kleine Hilfsmethode, um Bilder anzeigen zu können:

In [ ]:
from keras.preprocessing import image
def plots_idx(idx, path, filenames, titles=None):
    """Loads and displays images with given titles. The images are given with their index in filenames."""
    utils.plots([image.load_img(os.path.join(path,filenames[i])) for i in idx], titles=titles)

Wir laden die Ergebnisse aus den oben geschriebenen Dateien:

In [ ]:
predictions = utils.load_array(predictions_file)
filenames = utils.load_array(filenames_file)

Wir wählen zufällig ein paar Bilder aus und zeigen sie mit der Vorhersage an (`[Wahrscheinlichkeit_Katze, Wahrscheinlichkeit_Hund]`).

In [ ]:
idx = np.random.randint(0, len(test_batches.filenames),4)
plots_idx(idx, path=test_path, filenames=test_batches.filenames, titles=predictions[idx])

## Visualisieren der Ergebnisse

Wir wollen uns anschauen, wie gut unser Modell eigentlich vorhersagt. Die Idee dazu ist, dass wir mit dem Modell eine Vorhersage über die bereits klassifizierten Trainingsdaten machen. So kennen wir die "ground truth" zu jedem Bild und können ermitteln, ob die Vorhersage korrekt war.

Den folgenden Block muss man nur ausführen, wenn man das Training oben bereits vorher mal gemacht hatte und die Gewichte in eine Datei geschrieben hat. Dann kann man hier direkt das Modell neu laden mit den (veränderten) Gewichten. Hat man in der gleichen Sitzung das Training schon gemacht, kann man diesen Schritt überspringen.

In [ ]:
# Modell nochmal neu initialisieren (falls wir hier wieder beginnen wollen):
import vgg16
vgg = vgg16.Vgg16()
# Nicht vergessen, die letzte Schicht zu verändern (2 statt 1000 Klassen als Output)!
vgg.finetune(train_batches)
# Gewichte laden:
weights_filename = os.path.join(result_path,'finetune1.h5')
vgg.model.load_weights(weights_filename)

Vorhersage mit den Validierungsdaten. So kennnen wir die "ground truth" und können sie mit der Vorhersage des Modells vergleichen.

In [ ]:
# Vorhersage machen:
valid_batches, predictions = vgg.test(valid_path, batch_size=64)

print "First n predictions:"
print predictions[:8]

Wir stutzen uns das Ergebnis zurecht, so dass wir nur noch ein 1-dimeansionales Array mit einer `1` für einen vorhergesagten Hund und einer `0` für eine vorgesagte Katze haben:

In [ ]:
our_predictions = predictions[:,1]
print our_predictions[:8]
our_labels = np.round(our_predictions)
print "Prediction, if it's a dog: ", our_labels[:8]


Das ist unsere "ground truth":

In [ ]:
expected_labels = valid_batches.classes
filenames = valid_batches.filenames

### Zeige einige korrekte Klassifizierungen

Mit Numpy (`np`) können wir wir den Index aller Bilder ermitteln, bei denen unser vorhergesagtes Label (0 oder 1) mit dem erwarteten Label der "groud truth" übereinstimmt. Davon wählen wir zufällig 4 aus und zeigen sie an:

In [ ]:
correct = np.where(our_labels==expected_labels)[0]
print "Found {} correct labels".format(len(correct))
idx = np.random.permutation(correct)[:4]
titles = [filenames[i]+'\n'+ str(our_labels[i]) for i in idx]
plots_idx(idx, valid_path, filenames, titles)

### Zeige einige falsche Klassifizierungen

In [ ]:
incorrect = np.where(our_labels!=expected_labels)[0]
print "Found {} incorrect labels.".format(len(incorrect))
if len(incorrect)>0:
    idx = np.random.permutation(incorrect)[:4]
    titles = [filenames[i]+'\n'+ str(our_labels[i]) for i in idx]
    plots_idx(idx, valid_path, filenames, titles)

### Zeige einige richtige Klassifizierungen mit großer  Wahrscheinlichkeit

... also die, bei denen das Modell wirklich recht hatte.

In [ ]:
n_view = 4

# The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((our_labels==1) & (our_labels==expected_labels))[0]
print "Found {} confident correct dogs labels".format(len(correct_dogs))
most_correct_dogs = np.argsort(our_predictions[correct_dogs])[::-1][:n_view]
plots_idx(correct_dogs[most_correct_dogs], valid_path, filenames, our_predictions[correct_dogs][most_correct_dogs])

# The images we most confident were cats, and are actually cats
correct_cats = np.where((our_labels==0) & (our_labels==expected_labels))[0]
print "Found {} confident correct cats labels".format(len(correct_cats))
most_correct_cats = np.argsort(our_predictions[correct_cats])[::][:n_view]
plots_idx(correct_cats[most_correct_cats], valid_path, filenames, our_predictions[correct_cats][most_correct_cats])


### Zeige einige falsche Klassifizierungen mit der großer Wahrscheinlichkeit

... also die, bei denen das Modell total daneben lag.

In [ ]:
# The images we most confident were dogs, and are actually dogs
configent_dogs = np.where((our_labels==1) & (our_labels!=expected_labels))[0]
print "Found {} confident incorrect dogs labels".format(len(confident_dogs))
if len(confident_dogs)>0:
    most_confident_dogs = np.argsort(our_predictions[confident_dogs])[::-1][:n_view]
    plots_idx(confident_dogs[most_confident_dogs], valid_path, filenames, our_predictions[confident_dogs][most_confident_dogs])

#threshold = 0.9
#confident_dogs = np.where((our_labels!=expected_labels) & (our_labels==1) & (our_predictions>threshold))[0]
#print "Found {} confident incorrect dogs labels".format(len(confident_dogs))
#if len(confident_dogs)>0:
#    idx = np.random.permutation(confident_dogs)[:4]
#    titles = [str(our_predictions[i])+'\n'+filenames[i] for i in idx]
#    plots_idx(idx, valid_path, filenames, titles)

#threshold = 0.0001
#confident_cats = np.where((our_labels!=expected_labels) & (our_labels==0) & (our_predictions<threshold))[0]
#print "Found {} confident incorrect cats labels".format(len(confident_cats))
#if len(confident_cats)>0:
#    idx = np.random.permutation(confident_cats)[:4]
#    titles = [str(our_predictions[i])+'\n'+filenames[i] for i in idx]
#    plots_idx(idx,valid_path, filenames, titles)

# The images we most confident were dogs, and are actually dogs
configent_cats = np.where((our_labels==0) & (our_labels!=expected_labels))[0]
print "Found {} confident incorrect cats labels".format(len(confident_cats))
if len(confident_cats)>0:
    most_confident_cats = np.argsort(our_predictions[confident_cats])[::][:n_view]
    plots_idx(confident_cats[most_confident_cats], valid_path, filenames, our_predictions[confident_cats][most_confident_cats])


### Ziege die unsichersten Klassifizierungen

... also die, bei denen sich das Modell nicht so sicher war.

In [ ]:
uncertain = np.argsort(np.abs(our_predictions-0.5))
print our_predictions
print uncertain
#titles = [str(our_predictions[i])+'\n'+filenames[i] for i in uncertain]
plots_idx(uncertain[:6], valid_path, filenames, our_predictions)

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels,our_labels)
utils.plot_confusion_matrix(cm, valid_batches.class_indices)

## Mehr Layer trainieren

Bisher haben wir nur den letzten Layer des Modells neu trainiert und den Rest des Modells nicht angetastet. Wenn wir nun auch die mittleren Layer re-trainieren wollen geht das mit Keras ziemlich einfach...

Zunächst: Modell laden und initalisieren, indem wir die Gewichte aus unserem Training oben laden:

In [ ]:
# Modell nochmal neu initialisieren:
vgg = vgg16.Vgg16()
# Nicht vergessen, die letzte Schicht zu verändern (2 statt 1000 Klassen als Output)!
vgg.finetune(train_batches)
# Gewichte aus Datei laden:
weights_filename = os.path.join(result_path,'finetune1.h5')
vgg.model.load_weights(weights_filename)

Nochmal kurz einen Blick auf die Struktur des Modells werfen:

In [ ]:
vgg.model.summary()

Achtung: Wir müssen darauf achten, den letzten Layer (den wir oben selbst hinzugefügt haben) vorher über `vgg.fit()` auch trainiert zu haben, da er sonst mit Zufallswerten initialisert ist, welche das Training der Zwischenschichten ziemlich durcheinander bringen würde. Wenn wir die zuvor in einer Datei gespeicherten Gewichte verwenden ist das automatisch der Fall.


### Alle Dense Layer neu trainieren
Der erste Versuch ist, nur die `Dense` Layer am Ende des Modells neu zu trainieren. `Dense` Layer bilden Lineare Funktionen ab, die mit allen Outputs der vorigen Layer verbunden sind.

In [ ]:
# Hilfsmethode, um Modell anzupassen (fitting):
def fit_model(model, train_batches, validation_batches, nb_epoch=1):
    model.fit_generator(train_batches, samples_per_epoch=train_batches.N, nb_epoch=nb_epoch, 
                        validation_data=validation_batches, nb_val_samples=validation_batches.N)
    
# Hole den Index des ersten "dense" layers:
first_dense_idx = [index for index,layer in enumerate(vgg.model.layers) if type(layer) is keras.layers.core.Dense][0]
print "First dense layer is layer no. " + str(first_dense_idx)
# ...und setze diesen und alle nachfolgenden auf "trainierbar":
for layer in vgg.model.layers[first_dense_idx:]: layer.trainable=True

Jetzt trainieren wir _alle_ Layer ab dem ersten "Dense" Layer neu (diesmal mit 3 Durchläufen durch die Trainingsdaten) und speichern die Gewichte wieder:

In [ ]:
keras.backend.set_value(vgg.model.optimizer.lr, 0.01)
weights_filename = os.path.join(result_path,'finetune2.h5')

fit_model(vgg.model, train_batches, validation_batches, 3)

vgg.model.save_weights(weights_filename)

Um das Modell zu beurteilen, machen wir wieder eine Vorhersage über ein klassifziertes Dataset ("ground truth") und berechnen eine Cross Entropy Matrix:

In [ ]:
def predict_and_plot_confusion_matrix(vgg, path):
    # Prediction:
    batches, predictions = vgg.test(path, batch_size=64)
    our_predictions = predictions[:,0]
    our_labels = np.round(1-our_predictions)

    # Ground truth:
    expected_labels = batches.classes

    cm = sklearn.metrics.confusion_matrix(expected_labels,our_labels)
    utils.plot_confusion_matrix(cm, batches.class_indices)

In [ ]:
predict_and_plot_confusion_matrix(vgg,valid_path)

### Noch mehr Layer trainieren

Wir können auch versuchen, noch mehr Layer zu trainieren (nicht nur die Dense-Layer am hinteren Ende des Networks):

In [ ]:
for layer in vgg.model.layers[12:]: 
    layer.trainable=True

keras.backend.set_value(vgg.model.optimizer.lr, 0.001)
model_file = os.path.join(result_path,'finetune3.h5')

fit_model(vgg.model, train_batches, validation_batches, 4)

vgg.model.save_weights(model_file)

Auch hier berechnen wir wieder eine Confusion Matrix:

In [ ]:
predict_and_plot_confusion_matrix(vgg,valid_path)

## Exkurs: Wenige Bilder Trainieren, viele Bilder klassifizieren

Wie gut ist unser Modell eigentlich, wenn wir nur mit wenigen Bildern trainieren?

Vorgehensweise:
* Trainieren mit den Bildern des `sample` Datasets (200 Bilder)
* Vorhersagen mit den Bildern des normalen Datasets (23000 Trainingsbilder als Ground Truth)

Da wir hier nicht das ganze Training wiederholen wollen, laden wir die Gewichte aus dem `sample` Pfad - das setzt voraus, dass man den Trainings-Code oben auch mal mit dem `sample` Dataset ausgeführt hat und somit die Datei mit den Gewichten existiert. Im Code unten kann man festlegen, welche Gewichte wir verwenden wollen.

In [ ]:
# Modell nochmal neu initialisieren (falls wir hier wieder beginnen wollen):
vgg = vgg16.Vgg16()
# Nicht vergessen, die letzte Schicht zu verändern (2 statt 1000 Klassen als Output)!
vgg.finetune(train_batches)

# Gewichte laden (aus dem 'sample' Pfad!)
#weights = 'finetune1.h5'   # Nur der letzte Layer wurde trainiert
#weights = 'finetune2.h5'   # Alle Dense Layer wurden trainiert
weights = 'finetune3.h5'   # Alle Layer ab Nr. 12 wurden trainiert
weights_filename = os.path.join('sample','results',weights)

print "using weights " + weights_filename
vgg.model.load_weights(weights_filename)

In [ ]:
# Jetzt führen wir die Vorhersage mit dem "normalen" Dataset durch:
predict_and_plot_confusion_matrix(vgg,os.path.join('data','train'))

Das Modell hat also nur ~820 von 23.000 Bildern falsch klassifiziert (~3,6%), obwohl das Finetuning mit nur 200 Bildern erfolgte (nicht vergessen: Das Modell wurde mit den Bildern aus Imagnet vortrainiert!). Dieser Werte lässt sich sicher noch verbessern, wenn wir das Finetuning auch auf andere Layer ausweiten - wie im vorigen Abschnitt beschrieben.